import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import os

import findspark
findspark.init()
findspark.find()

'C:\\devprogs\\spark-3.3.1-bin-hadoop3'

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when
from pyspark.sql.functions import col, asc,desc

create local SparkSession

In [3]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

read csv with inferschema

In [4]:
df4 = spark.read.options(inferSchema='True',delimiter=',').csv("ds_salaries.csv")

read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [6]:
df4 = spark.read.options(inferSchema='True',delimiter=',').csv("ds_salaries.csv")

write schema of scv on screen

In [5]:
df4.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)



create schema of this scv

In [6]:
my_schema = StructType() \
      .add("ID",IntegerType(),True) \
      .add("work_year",IntegerType(),True) \
      .add("experience_level",StringType(),True) \
      .add("employment_type",StringType(),True) \
      .add("job_title",StringType(),True) \
      .add("salary",IntegerType(),True) \
      .add("salary_currency",StringType(),True) \
      .add("salary_in_usd",IntegerType(),True) \
      .add("employee_residence",StringType(),True) \
      .add("remote_ratio",IntegerType(),True) \
      .add("company_location",StringType(),True) \
      .add("company_size",StringType(),True)

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [7]:
df_with_schema = spark.read.format("csv") \
      .option("header", True) \
      .schema(my_schema) \
      .load("ds_salaries.csv")
df_with_schema.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [8]:
df_with_schema.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [9]:
df = df_with_schema
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| ID|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [10]:
display(df.toPandas())

,ID,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [11]:
df_job_title = df.dropDuplicates(["job_title"])

print all rows from df_job_titles without truncating jobs

In [12]:
df_job_title.show(truncate = False)

+---+---------+----------------+---------------+----------------------------------+-------+---------------+-------------+------------------+------------+----------------+------------+
|ID |work_year|experience_level|employment_type|job_title                         |salary |salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+----------------------------------+-------+---------------+-------------+------------------+------------+----------------+------------+
|77 |2021     |MI              |PT             |3D Computer Vision Researcher     |400000 |INR            |5409         |IN                |50          |IN              |M           |
|52 |2020     |EN              |FT             |AI Scientist                      |300000 |DKK            |45896        |DK                |50          |DK              |S           |
|344|2022     |EX              |FT             |Analytics Engineer              

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [13]:
df_analytic = df.groupBy("job_title") \
    .agg(min("salary_in_usd").alias("min_salary"), \
         avg("salary_in_usd").alias("avg_salary"), \
         max("salary_in_usd").alias("max_salary") \
     )

print all rows from df_analytic without trancating jobs

In [14]:
df_analytic.show(truncate=False)

+----------------------------------------+----------+------------------+----------+
|job_title                               |min_salary|avg_salary        |max_salary|
+----------------------------------------+----------+------------------+----------+
|3D Computer Vision Researcher           |5409      |5409.0            |5409      |
|Lead Data Engineer                      |56000     |139724.5          |276000    |
|Head of Machine Learning                |79039     |79039.0           |79039     |
|Data Specialist                         |165000    |165000.0          |165000    |
|Data Analytics Lead                     |405000    |405000.0          |405000    |
|Machine Learning Scientist              |12000     |158412.5          |260000    |
|Lead Data Analyst                       |19609     |92203.0           |170000    |
|Data Engineering Manager                |59303     |123227.2          |174000    |
|Staff Data Scientist                    |105000    |105000.0          |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [15]:
windowSpec  = Window.orderBy(desc('avg_salary'))
df_analytic = df_analytic.withColumn("row_id",row_number().over(windowSpec))

print all data from df_analytic

In [17]:
df_analytic.show(truncate=False)

+----------------------------------+----------+------------------+----------+------+
|job_title                         |min_salary|avg_salary        |max_salary|row_id|
+----------------------------------+----------+------------------+----------+------+
|Data Analytics Lead               |405000    |405000.0          |405000    |1     |
|Principal Data Engineer           |185000    |328333.3333333333 |600000    |2     |
|Financial Data Analyst            |100000    |275000.0          |450000    |3     |
|Principal Data Scientist          |148261    |215242.42857142858|416000    |4     |
|Director of Data Science          |130026    |195074.0          |325000    |5     |
|Data Architect                    |90700     |177873.9090909091 |266400    |6     |
|Applied Data Scientist            |54238     |175655.0          |380000    |7     |
|Analytics Engineer                |135000    |175000.0          |205300    |8     |
|Data Specialist                   |165000    |165000.0          

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [18]:
df_analytic = df_analytic.select('row_id', 'job_title','min_salary', 'max_salary', 'avg_salary')

print df_analytic now

In [19]:
df_analytic.show(truncate=False)

+------+----------------------------------+----------+----------+------------------+
|row_id|job_title                         |min_salary|max_salary|avg_salary        |
+------+----------------------------------+----------+----------+------------------+
|1     |Data Analytics Lead               |405000    |405000    |405000.0          |
|2     |Principal Data Engineer           |185000    |600000    |328333.3333333333 |
|3     |Financial Data Analyst            |100000    |450000    |275000.0          |
|4     |Principal Data Scientist          |148261    |416000    |215242.42857142858|
|5     |Director of Data Science          |130026    |325000    |195074.0          |
|6     |Data Architect                    |90700     |266400    |177873.9090909091 |
|7     |Applied Data Scientist            |54238     |380000    |175655.0          |
|8     |Analytics Engineer                |135000    |205300    |175000.0          |
|9     |Data Specialist                   |165000    |165000    |

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [20]:
df_exp_lvls = df.groupBy("experience_level").agg(max("salary_in_usd").alias("biggest_salary"))
df_exp_lvl = df.join(df_exp_lvls,df.experience_level == df_exp_lvls.experience_level,"inner").drop(df_exp_lvls.experience_level)

print here df_exp_lvl

In [21]:
df_exp_lvl.show(truncate=False)

+---+---------+---------------+--------------------------+--------+---------------+-------------+------------------+------------+----------------+------------+----------------+--------------+
|ID |work_year|employment_type|job_title                 |salary  |salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|experience_level|biggest_salary|
+---+---------+---------------+--------------------------+--------+---------------+-------------+------------------+------------+----------------+------------+----------------+--------------+
|0  |2020     |FT             |Data Scientist            |70000   |EUR            |79833        |DE                |0           |DE              |L           |MI              |450000        |
|1  |2020     |FT             |Machine Learning Scientist|260000  |USD            |260000       |JP                |0           |JP              |S           |SE              |412000        |
|2  |2020     |FT             |Big Data 

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [22]:
df_best = df_exp_lvl.filter(col("salary_in_usd") == col("biggest_salary"))
df_best = df_best.select('ID', 'experience_level', 'biggest_salary', 'employee_residence')

print df_best

In [23]:
df_best.show(truncate=False)

+---+----------------+--------------+------------------+
|ID |experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
|33 |MI              |450000        |US                |
|37 |EN              |250000        |US                |
|63 |SE              |412000        |US                |
|97 |MI              |450000        |US                |
|252|EX              |600000        |US                |
+---+----------------+--------------+------------------+



drop duplicates if exist by experience_level

In [24]:
df_best = df_best.dropDuplicates(["experience_level"])

print df_best

In [25]:
df_best.show(truncate=False)

+---+----------------+--------------+------------------+
|ID |experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
|37 |EN              |250000        |US                |
|252|EX              |600000        |US                |
|33 |MI              |450000        |US                |
|63 |SE              |412000        |US                |
+---+----------------+--------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [26]:
df_new_best = df_best.withColumn("exp_level", when(df_best.experience_level == "MI","we want middle")
                                 .when(df_best.experience_level == "SE","we want senior")
                                 .otherwise('Null'))
df_new_best = df_new_best.select('biggest_salary', 'employee_residence', "exp_level")

print df_new_best

In [27]:
df_new_best.show(truncate=False)

+--------------+------------------+--------------+
|biggest_salary|employee_residence|exp_level     |
+--------------+------------------+--------------+
|250000        |US                |Null          |
|600000        |US                |Null          |
|450000        |US                |we want middle|
|412000        |US                |we want senior|
+--------------+------------------+--------------+



write df_new_best like 1.csv and load then it to df_final

In [28]:
df_new_best.toPandas().to_csv('1.csv', header=True)

In [29]:
df_final = spark.read.options(header='True', inferSchema='True', delimiter=',') \
  .csv("1.csv")

print df_final

In [30]:
df_final.show(truncate=False)

+---+--------------+------------------+--------------+
|_c0|biggest_salary|employee_residence|exp_level     |
+---+--------------+------------------+--------------+
|0  |250000        |US                |Null          |
|1  |600000        |US                |Null          |
|2  |450000        |US                |we want middle|
|3  |412000        |US                |we want senior|
+---+--------------+------------------+--------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [31]:
df_final = df_final.filter(df_final.exp_level !='Null')
df_final_1 = df.join(df_final,df.salary_in_usd == df_final.biggest_salary,"inner").drop(df_final.biggest_salary)

print df_final

In [32]:
df_final_1.show(truncate=False)

+---+---------+----------------+---------------+----------------------+------+---------------+-------------+------------------+------------+----------------+------------+---+------------------+--------------+
|ID |work_year|experience_level|employment_type|job_title             |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|_c0|employee_residence|exp_level     |
+---+---------+----------------+---------------+----------------------+------+---------------+-------------+------------------+------------+----------------+------------+---+------------------+--------------+
|33 |2020     |MI              |FT             |Research Scientist    |450000|USD            |450000       |US                |0           |US              |M           |2  |US                |we want middle|
|63 |2020     |SE              |FT             |Data Scientist        |412000|USD            |412000       |US                |100         |US              |L      

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [33]:
biggest_salary = df_final_1.select(max('salary_in_usd'))
biggest_salary.show()

+------------------+
|max(salary_in_usd)|
+------------------+
|            450000|
+------------------+



It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system